In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null

In [0]:
!pip install keras-tcn

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
cd drive


In [0]:
cd My Drive

In [0]:
import keras
import os
os.chdir('/content/drive/My Drive/44Col/')
print(os.listdir())

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Fri Oct  5 16:23:15 2018

@author: Pooja
"""
#Meand and Cross Validation
import numpy as np
import os
import cv2
import keras
import time
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from keras.layers import Dense, Flatten, Dropout,Input, Bidirectional,BatchNormalization
#from keras.layers import Merge
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import TimeDistributed	
from keras.layers.convolutional import (Conv2D,MaxPooling2D)
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint	
from keras.models import load_model
from keras.layers import multiply
from keras.layers.core import *
from keras.models import *
max_seq = 44
num = np.zeros(6)
path_uiuc = '/content/drive/My Drive/'
x = list()
y = list()
data_fold = ['44D1', '44D2', '44Col']
for fold in data_fold:
    path_44 = path_uiuc + fold + '/'
    print(path_44)
    vidlist = os.listdir(path_44)
    it = len(vidlist)
    for i in range(it):
        s1 = vidlist[i]
        _, typ = s1.split('_')
        #NO REACT
        if typ == '3':
            y.append(3)
        
        elif typ == '0':
           y.append(0)
           
        elif typ == '1':
           y.append(1)
           
        elif typ == '2':
           y.append(2)
           
        elif typ == '4':   
           y.append(4)
           
        elif typ == '5':
           y.append(5)
        else:
            print('Error!' + str(typ))
            
        num[int(typ)] +=1
        
        os.chdir(path_44 + s1)   
        img_list = os.listdir()
        it0 = len(img_list)
        temp = list()
        dim = (32,32)
        for j in range(it0):
            img = cv2.imread(img_list[j])
            print(str(i)+'_'+str(j), end = ' ')
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, dim, interpolation = cv2.INTER_CUBIC)
            mn1 = np.mean(img)
            st1 = np.std(img)
            img = (img.copy() - mn1) / st1
            #img = img.copy().flatten()
            temp.append(img)
        temp = np.asarray(temp)     
        x.append(temp)
        
strokes = {'backhand':num[0], 'forehand':num[1], 'lob':num[2], 'react':num[3], 'serve':num[4], 'smash':num[5]}
for i in strokes.keys():
    print("{} - %d".format(i)%(strokes[i]))
    
x = np.asarray(x)
y = np.asarray(y)
print(x.shape)
x = x.reshape(x.shape[0], x.shape[1], x.shape[2], x.shape[3], 1)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42,shuffle =True)
X_train=np.asarray(X_train)
X_test=np.asarray(X_test)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
#y_train = np_utils.to_categorical(y_train, 6)
y_test = np_utils.to_categorical(y_test, 6)

seed = 10   
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvscores = []
ii = 0 

#model
x = Input((44,32,32,1))
model = (TimeDistributed(Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same')))(x)
model = (TimeDistributed(Conv2D(32, (3,3),  activation='relu')))(model)
model = (TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))(model)
model = (BatchNormalization())(model)
#model = (Dropout(0.25)(model)

model = (TimeDistributed(Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same')))(model)
model = (TimeDistributed(Conv2D(64, (3,3),  activation='relu')))(model)
model = (TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))(model)
model = (BatchNormalization())(model)
#model = (Dropout(0.25)(model)

model = (TimeDistributed(Flatten()))(model)

model = (Dropout(0.5))(model)
model = (Bidirectional(LSTM(512, return_sequences=False, dropout=0.4)))(model)
model = (Dense(6, activation='softmax'))(model)
model = Model(input = x, output = model)
print(model.summary())
start = time.time()
for train, test in kfold.split(X_train, y_train):
    print('------------ITERATION : ' + str(ii) + '  -------------------') 
	# Compile model
    #sgd = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    sgd = keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
	# Fit the model
    Yy = np_utils.to_categorical(y_train, 6)
    model.fit(X_train[train], Yy[train], epochs=80, batch_size=32, verbose=1)
    model.save('/content/drive/My Drive/conv_s_m'+str(ii)+'.h5')
    ii += 1
    #print(model.predict(X[train], batch_size=32))
	# evaluate the model
    scores = model.evaluate(X_train[test], Yy[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
exectime = time.time()-start    
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

final_scores = []
for i in range(1):
  mod = load_model('/content/drive/My Drive/conv_s_m'+str(i)+'.h5')
  eval1 = time.time()
  scores = model.evaluate(X_test, y_test, verbose=1)
  evaltim = time.time()-eval1
  print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
  final_scores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(final_scores), np.std(final_scores)))
print(exectime, evaltim)